In [1]:
import os
import main
import BuildModel
import json
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

from Embedding import FeatureExtractor

In [2]:
config = json.load(open('config.json', 'r'))
print(config)
model,history = main.run(config['embedding_vector'],verbose=False)

In [3]:
plt.plot(history['train_losses'])
plt.plot(history['eval_losses'])
plt.plot(history['train_f1s'])
plt.plot(history['eval_f1s'])
plt.show()

In [3]:
from Embedding import FeatureExtractor

In [6]:
import torch
torch.save(model.state_dict(), 'model.pt')

In [79]:
model = BuildModel.Resnet101(10,config['embedding_vector']).get_model()
model.load_state_dict(torch.load('model.pt'))
model.eval()

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

------------------------------------

In [146]:
from tqdm import tqdm
root = 'data/genres/'
all_waves = [root + genre + '/' + wav for genre in os.listdir(root) for wav in os.listdir(root + genre)]
wave2vec = {}
for wave_root in tqdm(all_waves):
    emb_vec, genre = extractor.feature(wave_root, config)
    wave2vec[wave_root] = emb_vec[0].detach().numpy(), genre

100%|██████████| 1000/1000 [05:09<00:00,  3.23it/s]


# ROCK의 3번째 곡과 가장 유사한 곡을 추천해줘~~(성공)

In [158]:
import numpy as np
import librosa
import IPython.display

In [181]:
my_root = 'data/genres/jazz/jazz.00020.wav'
extractor = FeatureExtractor(model)
feature , genre = extractor.feature(my_root,config)

In [182]:
data,sr = librosa.load(my_root)
IPython.display.Audio(data=data, rate=sr)

In [183]:
def get_nearest_music(feature,all_featrues):
    min_dist = 9999
    min_music = None
    for key in wave2vec:
        vec = wave2vec[key][0]
        dist = np.linalg.norm(feature.detach().numpy()-vec)
        if dist == 0:continue
        if dist < min_dist:
            min_dist = dist
            min_music = key
    return min_dist, min_music

In [184]:
dist, music = get_nearest_music(feature,wave2vec)
print(dist, music)

6.349371 data/genres/classical/classical.00037.wav


In [185]:
data,sr = librosa.load(music)
IPython.display.Audio(data=data, rate=sr)

# Elbow method를 통해 적절한 갯수의 군집 산정